# Silver to Gold Step

## Notebook Objective

In this notebook you:
1. Harden the Silver to Gold step we wrote in the previous notebook using the composable functions in the operations file.

## Step Configuration

In [0]:
%run ./includes/configuration

Out[23]: DataFrame[]

No running streams.


## Import Operation Functions

In [0]:
%run ./includes/main/python/operations

## Start Streams

Before we add new streams, let's start the streams we have previously engineered.

We will start two named streams:

- `write_raw_to_bronze`
- `write_bronze_to_silver`

### Current Delta Architecture

Next, we demonstrate everything we have built up to this point in our
Delta Architecture.

Again, we do so with composable functions included in the
file `includes/main/python/operations`.

In [0]:
rawDF = read_stream_raw(spark, rawPath)
transformedRawDF = transform_raw(rawDF)
rawToBronzeWriter = create_stream_writer(
    dataframe=transformedRawDF,
    checkpoint=bronzeCheckpoint,
    name="write_raw_to_bronze",
    partition_column="p_ingestdate",
)
rawToBronzeWriter.start(bronzePath)

bronzeDF = read_stream_delta(spark, bronzePath)
transformedBronzeDF = transform_bronze(bronzeDF)
bronzeToSilverWriter = create_stream_writer(
    dataframe=transformedBronzeDF,
    checkpoint=silverCheckpoint,
    name="write_bronze_to_silver",
    partition_column="p_eventdate",
)
bronzeToSilverWriter.start(silverPath)

Out[35]: <pyspark.sql.streaming.StreamingQuery at 0x7fb52162ebb0>

**Exercise:** Harden the Silver to Gold step that we created in the previous notebook.

Now that you have seen the pattern, fill out the following code block to complete this step.

💁🏻‍♀️Remember to use `mode="complete"` with streaming aggregate tables.

In [0]:
tableName = "/aggregate_heartrate"
tableCheckpoint = goldCheckpoint + tableName
tablePath = goldPath + tableName
 
silverDF = read_stream_delta(spark, silverPath)

transformedSilverDF = transform_silver_mean_agg(silverDF)

silverToGoldAggWriter = create_stream_writer(
    dataframe=transformedSilverDF,
    checkpoint=tableCheckpoint,
    name="write_silver_to_gold",
    mode="complete",
)
silverToGoldAggWriter.start(tablePath)

Out[36]: <pyspark.sql.streaming.StreamingQuery at 0x7fb520589220>

## **Exercise:** Show all running streams

In [0]:

for stream in spark.streams.active:
    print(stream.name)

write_bronze_to_silver
write_raw_to_bronze
write_silver_to_gold


## Stop All Streams

In the next notebook, we will take a look at schema enforcement and evolution with Delta Lake.

Before we do so, let's shut down all streams in this notebook.

In [0]:
stop_all_streams()


Out[38]: True

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>